In [7]:
CREATE DATABASE spotify;

Commands completed successfully.

Total execution time: 00:00:00.521

## **1- TAM, ARTIK, FARK YEDEKLEMELERİ**

## **_<u>Tüm Veritabanını (Full Backup) Yedekleme</u>_**

In [1]:
BACKUP DATABASE spotify
TO DISK = '/var/opt/mssql/backup/spotify_full.bak'
WITH INIT,
     NAME = 'Full Backup of spotify',
     STATS = 10;

11 percent processed.

21 percent processed.

31 percent processed.

41 percent processed.

51 percent processed.

...

100 percent processed.

Processed 416 pages for database 'spotify', file 'spotify' on file 1.

Processed 2 pages for database 'spotify', file 'spotify_log' on file 1.

BACKUP DATABASE successfully processed 418 pages in 0.051 seconds (63.955 MB/sec).

Total execution time: 00:00:00.162

## **_<u>Artık Yedeklemeler (Transaction Log Backup)</u>_**

In [6]:
ALTER DATABASE spotify SET RECOVERY FULL;

Commands completed successfully.

Total execution time: 00:00:00.037

In [8]:
SELECT name, recovery_model_desc
FROM sys.databases
WHERE name = 'spotify';

(1 row affected)

Total execution time: 00:00:00.010

name,recovery_model_desc
spotify,FULL


In [13]:
BACKUP LOG spotify
TO DISK = '/var/opt/mssql/backup/spotify_log.trn'
WITH INIT,
     NAME = 'First Transaction Log Backup of spotify',
     STATS = 10;

100 percent processed.

Processed 2 pages for database 'spotify', file 'spotify_log' on file 1.

BACKUP LOG successfully processed 2 pages in 0.005 seconds (2.343 MB/sec).

Total execution time: 00:00:00.125

In [9]:
BACKUP LOG spotify
TO DISK = '/var/opt/mssql/backup/spotify_log.trn'
WITH INIT,
     NAME = 'Transaction Log Backup of spotify',
     STATS = 10;

: Msg 4214, Level 16, State 1, Line 1
BACKUP LOG cannot be performed because there is no current database backup.

: Msg 3013, Level 16, State 1, Line 1
BACKUP LOG is terminating abnormally.

Total execution time: 00:00:00.027

## **_<u>Fark Yedekleme (Differential Backup)</u>_**

In [11]:
BACKUP DATABASE spotify
TO DISK = '/var/opt/mssql/backup/spotify_diff.bak'
WITH DIFFERENTIAL,
     INIT,
     NAME = 'Differential Backup of spotify',
     STATS = 10;


10 percent processed.

21 percent processed.

32 percent processed.

43 percent processed.

54 percent processed.

...

100 percent processed.

Processed 160 pages for database 'spotify', file 'spotify' on file 1.

Processed 2 pages for database 'spotify', file 'spotify_log' on file 1.

BACKUP DATABASE WITH DIFFERENTIAL successfully processed 162 pages in 0.050 seconds (25.234 MB/sec).

Total execution time: 00:00:00.244

## **_2-_ ZAMANLAYICI İLE YEDEKLEME OTOMASYONU (Scheduled Backup Job)**

In [15]:
SELECT servicename, status_desc
FROM sys.dm_server_services
WHERE servicename LIKE '%SQL Server Agent%';

(1 row affected)

Total execution time: 00:00:00.053

servicename,status_desc
SQL Server Agent (MSSQLSERVER),Running


**SQL Server Agent ile devam etmek için** <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"><b>Server Agent Extension ile</b></span> **yeni bir Job adımı oluşturacağız.**

In [16]:
USE msdb;
GO

EXEC dbo.sp_add_job
    @job_name = N'SpotifyDB_Backup_Job';
GO

EXEC sp_add_jobstep
    @job_name = N'SpotifyDB_Backup_Job',
    @step_name = N'Backup Spotify Database',
    @subsystem = N'TSQL',
    @command = N'BACKUP DATABASE [spotify] TO DISK = ''/var/opt/mssql/backups/spotify_$(ESCAPE_SQUOTE(DATE)).bak'' WITH COMPRESSION, STATS = 10',
    @database_name = N'master';
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.269

In [17]:
EXEC dbo.sp_add_schedule
    @schedule_name = N'Daily_8AM',
    @freq_type = 4, -- Daily
    @freq_interval = 1, -- Every day
    @active_start_time = 080000; -- 8:00 AM
GO

EXEC sp_attach_schedule
    @job_name = N'SpotifyDB_Backup_Job',
    @schedule_name = N'Daily_8AM';
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.142

In [18]:
EXEC dbo.sp_add_jobserver
    @job_name = N'SpotifyDB_Backup_Job';
GO

Commands completed successfully.

Total execution time: 00:00:00.322

## **_<span style="font-size: 16.002px; background-color: rgb(255, 255, 255);">Docker Volume ile yedeklerin kalıcı olması sağlanıyor.</span>_**  

<span style="font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">docker</span> <span style="color: rgb(73, 73, 73); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);"> run </span> <span class="token parameter variable" style="color: rgb(64, 120, 242); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">-e</span> <span class="token string" style="color: rgb(80, 161, 79); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">"MSSQL_AGENT_ENABLED=true"</span> <span class="token parameter variable" style="color: rgb(64, 120, 242); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">-d</span> <span class="token parameter variable" style="color: rgb(64, 120, 242); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">-p</span> <span class="token number" style="color: rgb(183, 107, 1); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">1433</span><span style="color: rgb(73, 73, 73); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">:1433 </span> <span class="token parameter variable" style="color: rgb(64, 120, 242); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">--name</span> <span style="color: rgb(73, 73, 73); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);"> sql_server_container </span> <span class="token parameter variable" style="color: rgb(64, 120, 242); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">-e</span> <span class="token string" style="color: rgb(80, 161, 79); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">"ACCEPT_EULA=Y"</span> <span class="token parameter variable" style="color: rgb(64, 120, 242); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">-e</span> <span class="token string" style="color: rgb(80, 161, 79); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">"SA_PASSWORD=MyStrongPass123"</span> <span class="token parameter variable" style="color: rgb(64, 120, 242); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);">-v</span> <span style="color: rgb(73, 73, 73); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 12.573px; white-space-collapse: preserve; background-color: rgb(250, 250, 250);"> sql_backups:/var/opt/mssql/backups mcr.microsoft.com/mssql/server:2019-latest</span>

### **Alternatif olarak aynı işlemi Cron Job ile de yapabiliriz. Aşağıdaki kodu "<span style="color: rgb(64, 64, 64); font-family: Menlo, &quot;Roboto Mono&quot;, &quot;Courier New&quot;, Courier, monospace, Inter, sans-serif; font-size: 14.0018px; background-color: rgb(236, 236, 236);">/usr/local/bin/backup_spotify.sh</span>" dosyası olarak kaydedeceğiz.**

_CONTAINER\_NAME="sql\_server\_container"_

_DB\_USER="sa"_

_DB\_PASS="MyStrongPass123”_

_DB\_NAME="spotify"_

_BACKUP\_DIR="/var/opt/mssql/backups/"_

_BACKUP\_FILE="${BACKUP\_DIR}/spotify\_$(date +%Y%m%d\_%H%M%S).bak"_

_docker exec $CONTAINER\_NAME /opt/mssql-tools/bin/sqlcmd -S localhost -U $DB\_USER -P $DB\_PASS -Q "BACKUP DATABASE \[$DB\_NAME\] TO DISK = N'/var/opt/mssql/backups/spotify.bak' WITH COMPRESSION, STATS = 10"_

_docker cp $CONTAINER\_NAME:/var/opt/mssql/backups/spotify.bak $BACKUP\_FILE_

_find $BACKUP\_DIR -name "spotify\_\*.bak" -type f -mtime +30 -exec rm {} \\;_

_echo "Backup completed: $BACKUP\_FILE"_

**Terminalimize de sırasıyla şu adımları yazacağız.**

- chmod +x /usr/local/bin/backup\_spotify.sh
- crontab -e
- 0 8 \* \* \* /usr/local/bin/backup\_spotify.sh \>\> /var/log/spotify\_backup.log 2\>&1

**<u>Gerçekleştirilen Cron Job işlemlerine ait Google Drive Dosyasının linki : [https://drive.google.com/drive/folders/1FT7GK8yYlzd8SG7d3MNLk-71x0EzUuSy?usp=sharing](https://drive.google.com/drive/folders/1FT7GK8yYlzd8SG7d3MNLk-71x0EzUuSy?usp=sharing)</u>**

In [26]:
SELECT 
    database_name,
    backup_start_date,
    backup_finish_date,
    type,
    physical_device_name,
    backup_size/1024/1024 AS backup_size_mb
FROM msdb.dbo.backupset bs
JOIN msdb.dbo.backupmediafamily bmf ON bs.media_set_id = bmf.media_set_id
WHERE database_name = 'spotify'
ORDER BY backup_start_date DESC;

(4 rows affected)

Total execution time: 00:00:00.029

database_name,backup_start_date,backup_finish_date,type,physical_device_name,backup_size_mb
spotify,2025-04-23 12:01:43.000,2025-04-23 12:01:43.000,D,/var/opt/mssql/backups/spotify_20250423.bak,3.39843750000
spotify,2025-04-23 11:49:23.000,2025-04-23 11:49:23.000,L,/var/opt/mssql/backup/spotify_log.trn,0.07812500000
spotify,2025-04-23 11:49:04.000,2025-04-23 11:49:04.000,I,/var/opt/mssql/backup/spotify_diff.bak,1.33593750000
spotify,2025-04-23 11:40:44.000,2025-04-23 11:40:44.000,D,/var/opt/mssql/backup/spotify_full.bak,3.33593750000


## **3- FELAKETTEN KURTARMA SENARYOLARI**

In [32]:
DROP DATABASE spotify;

Commands completed successfully.

Total execution time: 00:00:00.114

In [28]:
-- Veritabanındaki tüm bağlantıları sonlandır
USE master;
GO
ALTER DATABASE spotify SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
GO

-- Şimdi veritabanını silebilirsiniz
DROP DATABASE spotify;
GO

Commands completed successfully.

Nonqualified transactions are being rolled back. Estimated rollback completion: 0%.

Nonqualified transactions are being rolled back. Estimated rollback completion: 100%.

Commands completed successfully.

Total execution time: 00:00:03.222

In [33]:
RESTORE DATABASE spotify
FROM DISK = '/var/opt/mssql/backup/spotify_full.bak'
WITH REPLACE, NORECOVERY;

Processed 416 pages for database 'spotify', file 'spotify' on file 1.

Processed 2 pages for database 'spotify', file 'spotify_log' on file 1.

RESTORE DATABASE successfully processed 418 pages in 0.022 seconds (148.259 MB/sec).

Total execution time: 00:00:00.484

In [34]:
RESTORE DATABASE spotify
FROM DISK = '/var/opt/mssql/backup/spotify_diff.bak'
WITH NORECOVERY;

Processed 160 pages for database 'spotify', file 'spotify' on file 1.

Processed 2 pages for database 'spotify', file 'spotify_log' on file 1.

RESTORE DATABASE successfully processed 162 pages in 0.016 seconds (78.857 MB/sec).

Total execution time: 00:00:00.456

In [35]:
RESTORE LOG spotify
FROM DISK = '/var/opt/mssql/backup/spotify_log.trn'
WITH NORECOVERY;

Processed 0 pages for database 'spotify', file 'spotify' on file 1.

Processed 2 pages for database 'spotify', file 'spotify_log' on file 1.

RESTORE LOG successfully processed 2 pages in 0.024 seconds (0.488 MB/sec).

Total execution time: 00:00:00.279

In [36]:
RESTORE DATABASE spotify
WITH RECOVERY;

RESTORE DATABASE successfully processed 0 pages in 0.366 seconds (0.000 MB/sec).

Total execution time: 00:00:00.386

In [37]:
EXEC msdb.dbo.sp_help_job;

Commands completed successfully.

Total execution time: 00:00:00.217

job_id,originating_server,name,enabled,description,start_step_id,category,owner,notify_level_eventlog,notify_level_email,notify_level_netsend,notify_level_page,notify_email_operator,notify_netsend_operator,notify_page_operator,delete_level,date_created,date_modified,version_number,last_run_date,last_run_time,last_run_outcome,next_run_date,next_run_time,next_run_schedule_id,current_execution_status,current_execution_step,current_retry_attempt,has_step,has_schedule,has_target,type
0933f684-b0f2-4976-9a27-e6a3bfb5ebcb,D2AC8703F7AC,SpotifyDB_Backup_Job,1,No description available.,1,[Uncategorized (Local)],sa,2,0,0,0,(unknown),(unknown),(unknown),0,2025-04-23 11:52:23.697,2025-04-23 11:52:23.863,2,20250423,120143,1,20250423,150000,9,4,0 (unknown),0,1,2,1,1


In [38]:
EXEC msdb.dbo.sp_update_job
    @job_name = N'SpotifyDB_Backup_Job',
    @enabled = 1;

Commands completed successfully.

Total execution time: 00:00:00.159

**Point-in-time Restore**

In [40]:
RESTORE DATABASE spotify
FROM DISK = '/var/opt/mssql/backup/spotify_full.bak'
WITH STOPAT = '2025-04-23 12:00:00', RECOVERY;

Total execution time: 00:00:00